In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler


# Define the directory containing the saved results
results_dir = r"D:\Class Project\209\results"
output_dir = os.path.join(results_dir, "images_tiff")
os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
scaler = MinMaxScaler()


# Get a list of indices by finding unique files in the directory
num_samples = len([f for f in os.listdir(results_dir) if f.startswith("undersampled_")])

# Iterate through all the saved samples
for index in range(num_samples):
    # Load results for the current sample
    undersampled = np.load(os.path.join(results_dir, f"undersampled_{index}.npy"))
    output = np.load(os.path.join(results_dir, f"output_{index}.npy"))
    ground_truth = np.load(os.path.join(results_dir, f"ground_truth_{index}.npy"))
    
    undersampled = undersampled/np.max(undersampled)
    output = output/np.max(output)
    
    print(output.max())
    print(output.min())
    print(ground_truth.max())
    print(ground_truth.min())

    # Plot the results
    plt.figure(figsize=(16, 4))
    plt.subplot(1, 6, 1)
    plt.title("Undersampled")
    plt.imshow(undersampled, cmap='gray')
    plt.subplot(1, 6, 2)
    plt.title("Reconstructed")
    plt.imshow(output, cmap='gray')
    plt.subplot(1, 6, 3)
    plt.title("Ground Truth")
    plt.imshow(ground_truth, cmap='gray')
    plt.subplot(1, 6, 4)
    plt.title("Residual initial")
    plt.imshow(undersampled-ground_truth, cmap='gray')
    plt.subplot(1, 6, 5)
    plt.title("Residual after")
    plt.imshow(output-ground_truth, cmap='gray')
    plt.subplot(1, 6, 6)
    plt.title("Residual difference")
    plt.imshow(output-undersampled, cmap='gray')

    # Save the plot as a TIFF file
    save_path = os.path.join(output_dir, f"sample_{index}.tiff")
    plt.savefig(save_path, format='tiff')
    plt.close()  # Close the plot to free memory
    print(f"Saved plot for sample {index} as {save_path}")


1.0
-0.70814186
1.0
0.0
Saved plot for sample 0 as D:\Class Project\209\results\images_tiff\sample_0.tiff
1.0
-0.54999495
1.0
0.0
Saved plot for sample 1 as D:\Class Project\209\results\images_tiff\sample_1.tiff
1.0
-0.43979725
1.0
0.0
Saved plot for sample 2 as D:\Class Project\209\results\images_tiff\sample_2.tiff
1.0
-0.35559154
1.0
0.0
Saved plot for sample 3 as D:\Class Project\209\results\images_tiff\sample_3.tiff
1.0
-0.30829734
1.0
0.0
Saved plot for sample 4 as D:\Class Project\209\results\images_tiff\sample_4.tiff
1.0
-0.53814805
1.0
0.0
Saved plot for sample 5 as D:\Class Project\209\results\images_tiff\sample_5.tiff
1.0
-0.4484689
1.0
0.0
Saved plot for sample 6 as D:\Class Project\209\results\images_tiff\sample_6.tiff
1.0
-0.5078724
1.0
0.0
Saved plot for sample 7 as D:\Class Project\209\results\images_tiff\sample_7.tiff
1.0
-0.44205317
1.0
0.0
Saved plot for sample 8 as D:\Class Project\209\results\images_tiff\sample_8.tiff
1.0
-0.37033123
1.0
0.0
Saved plot for sample 9 